# 用机器学习方法完成中文文本分类

### 文本数据特征工程方法

- BOW/词袋模型
- TF-IDF
- word2vec

### 文本分类模型

- NB/SVM/GBDT
- Fasttext
- CNN/LSTM

# 朴素贝叶斯

我们试试用朴素贝叶斯完成一个中文文本分类器，一般在数据量足够，数据丰富度够的情况下，用朴素贝叶斯完成这个任务，准确度还是不错的。

机器学习的算法要取得好效果，离不开数据，我们先把数据加载进来看看。

### 准备数据

准备好数据，我们挑选 科技、汽车、娱乐、军事、运动 总共5类文本数据进行处理。

In [1]:
import jieba
import pandas as pd

# 科技
df_technology = pd.read_csv("./data/technology_news.csv", encoding='utf-8')
df_technology = df_technology.dropna()

# 汽车
df_car = pd.read_csv("./data/car_news.csv", encoding='utf-8')
df_car = df_car.dropna()

# 娱乐
df_entertainment = pd.read_csv("./data/entertainment_news.csv", encoding='utf-8')
df_entertainment = df_entertainment.dropna()

# 军事
df_military = pd.read_csv("./data/military_news.csv", encoding='utf-8')
df_military = df_military.dropna()

# 体育
df_sports = pd.read_csv("./data/sports_news.csv", encoding='utf-8')
df_sports = df_sports.dropna()

technology = df_technology.content.values.tolist()[1000:21000]
car = df_car.content.values.tolist()[1000:21000]
entertainment = df_entertainment.content.values.tolist()[:20000]
military = df_military.content.values.tolist()[:20000]
sports = df_sports.content.values.tolist()[:20000]

随便挑几条数据看看

In [2]:
print(technology[12])

　　现在家里都拉了网线，都能无线上网，一定要帮他们先登上WiFi，另外，老人不懂得流量是什么，也不知道如何开关，控制流量，所以设置好流量上限很重要，免得不小心点开了视频或者下载，电话费就大发了。


In [3]:
print(car[100])

　　截至发稿时，人人车给出的处理方案仍旧是检修车辆。王先生则认为，车辆在购买时就存在问题，但交易平台并未能检测出来。因此，王先生希望对方退款。王先生称，他将找专业机构对车辆进行鉴定，并通过法律途径维护自己的权益。J256


In [4]:
print(entertainment[10])

　　网综尺度相对较大原本是制作优势，《奇葩说》也曾经因为讨论的话题较为前卫一度引发争议。但《奇葩说》对于价值观的把握和引导让其中内含的“少儿不宜”只能算是小花絮。而纯粹是为了制造话题而“污”得“无节操无下限”的网综不仅让人生厌，也给节目自身招致了下架的厄运。对资本方而言，即便只从商业运营考量，点击量也分有价值和无价值，节目内容的变现能力如果建立在博眼球和低趣味迎合上，商业运营也难长久。对节目制作方与平台来说，为博一时的高点击而不顾底线不仅是砸自己招牌，以噱头吸引而来的观众与流量也是难以维持。


In [5]:
print(military[10])

　　央视记者 胡善敏：我现在所处的位置是在辽宁舰的飞行甲板，执行跨海区训练和试验任务的辽宁舰官兵，正在展开多个科目的训练，穿着不同颜色服装的官兵在紧张的对舰载机进行转运。


In [6]:
print(sports[10])

　　据统计，2016年仅在中国田径协会注册的马拉松赛事便达到了328场，继续呈现出爆发式增长的态势，2015年，这个数字还仅仅停留在134场。如果算上未在中国田协注册的纯“民间”赛事，国内全年的路跑赛事还要更多。


### 分词与中文文本处理

#### 停用词

In [7]:
stopwords = pd.read_csv("data/stopwords.txt",
                        index_col = False,
                        quoting = 3,
                        sep = "\t",
                        names = ['stopword'],
                        encoding = 'utf-8'
                       )
stopwords = stopwords['stopword'].values

#### 去停用词

In [8]:
def preprocess_text(content_lines, sentences, category):
    for line in content_lines:
        try:
            segs = jieba.lcut(line) # jieba分词，存入一个list中
            segs = filter(lambda x:len(x)>1, segs) # 只保留长度大于1的词
            segs = filter(lambda x:x not in stopwords, segs) # 只保留非停用词
            sentences.append((" ".join(segs), category)) # 用空格拼接过滤好的词，并在每个词后写上类别category
        except Exception as e:
            print(line)
            continue
            
# 生成训练数据
sentences = []

preprocess_text(technology, sentences, 'technology')
preprocess_text(car, sentences, 'car')
preprocess_text(entertainment, sentences, 'entertainment')
preprocess_text(military, sentences, 'military')
preprocess_text(sports, sentences, 'sports')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.425 seconds.
Prefix dict has been built succesfully.


#### 生成训练集

我们打乱一下顺序，生成更可靠的训练集

In [27]:
import random
random.shuffle(sentences)

In [28]:
# 输出一下前10个句子
for sentence in sentences[:10]:
    print(sentence[0], sentence[1])

苏宁 青创园 专注 青年 电商 文化 创意 影视制作 动漫 二次元 互联网 消费 升级 领域 项目 孵化 创业者 提供 完备 创业 环境 园区 一期 项目 占地约 9000 平方米 包含 四栋 众创 空间 两栋 商业 配套 录音棚 摄影棚 直播间 专业 配套 设施 基础 环境 服务 资源 生态 支持 青创园 背靠 苏宁 六大 产业 平台 平台 背后 更是 汇聚 用户 十万家 合作伙伴 完善 产业 服务 生态圈 technology
当晚 建安 飞机 顾不上 车旅 劳顿 直奔 医院 详细 患者 病情 第二天 一早 医生 交流 提出 治疗 建议 讨论 修改 治疗 方案 鼓励 安慰 患者 离开 医院 建安 许诺 电话 随叫随到 这名 边防 干部 康复 出院 military
人物 不知 拥护 爱戴 崇仰 国家 希望 奴隶 之邦 这句 名言 一代人 牢牢记 military
未来 中车 制造 停靠 波士顿 久负盛名 哈佛大学 麻省理工学院 走过 密歇根湖 芝加哥 抵达 洛杉矶 影星 荟萃 好莱坞 穿行 费城 世界 发达 市场 通向 世界 窗口 城市 展示 中国 风采 car
马化腾 去年 腾讯 电信 网络 诈骗 联合 大会 企业 个人信息 保护 视为 一项 工作 腾讯 重视 用户 信息安全 用户 个人信息 谨慎 采集 妥善 运营 保护 用户 知情权 数据 设置 防护 标准 专业 团队 采用 纵深 防御 理念 网络 主机 层面 相关 加固 措施 确保 用户 个人信息 马化腾 建议 提出 希望 政府 主管部门 牵头 行业 构建 个人信息 分级分类 保护 体系 完善 相关 岗位 工作人员 规范 管理 监督 technology
采写 京报 实习生 邵程 发自 乌镇 entertainment
本剧 不靠 天价 大腕 吸引 眼球 大胆 启用 一批 充满活力 年轻 演员 担任 主角 王海燕 冯国强 老戏骨 坐镇 甘当 绿叶 entertainment
X70 搭载 最强 智能 电视 旗舰 芯片 Mstar6A938 1.7 GHz 处理器 32GB 存储配置 性能 强劲 需求 系统 搭载 最新 EUI 5.9 拥有 分众 运营 个性 推荐 超级 语音 智能 导视 功能 蓝牙 4.1 802.11 ac 双频 Wi Fi USB3.0 接口 用户 连接 设备 音响 搭载 哈曼 卡顿 专

为了一会儿检测一下咱们的分类器效果怎么样，我们需要一份测试集。

所以把原数据集分成训练集的测试集，咱们用sklearn自带的分割函数。

In [29]:
from sklearn.model_selection import train_test_split
x, y = zip(*sentences)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1234)

In [30]:
len(x_train)

65696

下一步要做的就是在降噪数据上抽取出有用的特征啦，我们对文本抽取词袋模型特征

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(
    analyzer='word', # tokenise by character ngrams 解析器：基于空格来区分不同的词
    max_features=4000, # keep the most common 4000 ngrams 保留最高频的4000个词
)
vec.fit(x_train)

def get_features(x):
    vec.transform(x)

把分类器import进来，并训练

In [32]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(vec.transform(x_train), y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

看看我们的准确率如何

In [33]:
classifier.score(vec.transform(x_test), y_test)

0.840860313256313

In [34]:
len(x_test)

21899

我们可以看到，在2w多个样本上，我们能在5个类别上达到84的准确率。

有没有办法提高一些准确率呢？

我们可以把特征做得更棒一点，<br>
比如，试试加入抽取2-gram和3-gram的统计特征；<br>
比如，可以把词库的量放大一点。

In [35]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(
    analyzer='word', # tokenise by character ngrams 解析器：基于空格来区分不同的词
    ngram_range=(1,4), # use ngrams of size 1、2、3 保留单个词，两两词、三三词
    max_features=20000, # keep the most common 20000 ngrams 保留最高频的20000个词
)
vec.fit(x_train)

def get_features(x):
    vec.transform(x)

### 分类训练

In [36]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(vec.transform(x_train), y_train)
classifier.score(vec.transform(x_test), y_test)

0.8807251472669985

### 交叉验证

更可靠的验证效果的方式是交叉验证，但是交叉验证最好保证每一份里面的样本类别也是相对均衡的，我们这里使用StratifiedKFold（k折交叉验证）

In [41]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score
import numpy as np

# 分层抽样，保证样本的均衡性
def stratifiedkfold_cv(x, y, clf_class, shuffle=True, n_fold=5, **kwargs):
    stratifiedk_fold = StratifiedKFold(y, n_folds=n_fold, shuffle=shuffle)
    y_pred = y[:]
    for train_index, test_index in stratifiedk_fold:
        X_train, X_test = x[train_index], x[test_index]
        y_train = y[train_index]
        clf = clf_class(**kwargs)
        clf.fit(X_train, y_train)
        y_pred[test_index] = clf.predict(X_test)
    return y_pred

NB = MultinomialNB
print(precision_score(y, stratifiedkfold_cv(vec.transform(x), np.array(y), NB), average='macro'))

0.8806864356231184


我们做完K折的交叉验证，可以看到在5个类别上的结果平均准确度约为88%

### 我们自己来完成一个文本分类器class

In [42]:
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB


class TextClassifier():

    def __init__(self, classifier=MultinomialNB()):
        self.classifier = classifier
        self.vectorizer = CountVectorizer(analyzer='word', ngram_range=(1,4), max_features=20000)

    def features(self, X):
        return self.vectorizer.transform(X)

    def fit(self, X, y):
        self.vectorizer.fit(X)
        self.classifier.fit(self.features(X), y)

    def predict(self, x):
        return self.classifier.predict(self.features([x]))

    def score(self, X, y):
        return self.classifier.score(self.features(X), y)


In [43]:
text_classifier = TextClassifier()
text_classifier.fit(x_train, y_train)
#print(text_classifier.predict('这 是 有史以来 最 大 的 一 次 军舰 演习'))
print(text_classifier.predict('苹果 公司 有 新 的 发布 计划'))
print(text_classifier.score(x_test, y_test))

['technology']
0.8807251472669985


### SVM文本分类

我们来试试支持向量机的作用

In [45]:
from sklearn.svm import SVC
svm = SVC(kernel='linear')
svm.fit(vec.transform(x_train), y_train)
svm.score(vec.transform(x_test), y_test)

0.8464770080825609

我们可以试试rbf核

In [54]:
from sklearn.svm import SVC
svm = SVC(kernel='rbf')
svm.fit(vec.transform(x_train), y_train)
svm.score(vec.transform(x_test), y_test)

0.512306498013608

### 换特征/模型试试

In [47]:
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC


class TextClassifier():

    def __init__(self, classifier=SVC(kernel='linear')):
        self.classifier = classifier
        self.vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,3), max_features=12000)

    def features(self, X):
        return self.vectorizer.transform(X)

    def fit(self, X, y):
        self.vectorizer.fit(X)
        self.classifier.fit(self.features(X), y)

    def predict(self, x):
        return self.classifier.predict(self.features([x]))

    def score(self, X, y):
        return self.classifier.score(self.features(X), y)

In [53]:
text_classifier = TextClassifier()
text_classifier.fit(x_train, y_train)
print(text_classifier.predict('这 是 有史以来 最 大 的 一 次 军舰 演习'))
print(text_classifier.score(x_test, y_test))

['military']
0.8778026393899265
